# Leader Persona LoRA Training Notebook

This notebook walks through:
1. Installing dependencies
2. Extracting Leader messages from WhatsApp export
3. Converting to JSONL for instruction-tuning
4. Setting up model + tokenizer + LoRA
5. Preparing dataset
6. Training
7. Inference


In [1]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121 
!pip install transformers peft bitsandbytes datasets accelerate

Looking in indexes: https://download.pytorch.org/whl/cu121
  Using cached https://download.pytorch.org/whl/cu121/torch-2.5.1%2Bcu121-cp311-cp311-win_amd64.whl (2449.4 MB)
  Using cached https://download.pytorch.org/whl/cu121/torchvision-0.20.1%2Bcu121-cp311-cp311-win_amd64.whl (6.1 MB)
  Using cached https://download.pytorch.org/whl/cu121/torchaudio-2.5.1%2Bcu121-cp311-cp311-win_amd64.whl (4.1 MB)
  Using cached https://download.pytorch.org/whl/filelock-3.13.1-py3-none-any.whl.metadata (2.8 kB)
  Using cached https://download.pytorch.org/whl/networkx-3.3-py3-none-any.whl.metadata (5.1 kB)
  Using cached https://download.pytorch.org/whl/Jinja2-3.1.4-py3-none-any.whl.metadata (2.6 kB)
  Using cached https://download.pytorch.org/whl/fsspec-2024.6.1-py3-none-any.whl.metadata (11 kB)
  Using cached https://download.pytorch.org/whl/sympy-1.13.1-py3-none-any.whl (6.2 MB)
  Using cached https://download.pytorch.org/whl/mpmath-1.3.0-py3-none-any.whl (536 kB)
  Using cached https://download.pyto

  Using cached transformers-4.52.4-py3-none-any.whl.metadata (38 kB)
  Using cached peft-0.15.2-py3-none-any.whl.metadata (13 kB)
  Using cached bitsandbytes-0.46.0-py3-none-win_amd64.whl.metadata (10 kB)
  Using cached datasets-3.6.0-py3-none-any.whl.metadata (19 kB)
  Using cached accelerate-1.8.0-py3-none-any.whl.metadata (19 kB)
  Using cached huggingface_hub-0.33.0-py3-none-any.whl.metadata (14 kB)
  Using cached PyYAML-6.0.2-cp311-cp311-win_amd64.whl.metadata (2.1 kB)
  Using cached regex-2024.11.6-cp311-cp311-win_amd64.whl.metadata (41 kB)
  Using cached requests-2.32.4-py3-none-any.whl.metadata (4.9 kB)
  Using cached tokenizers-0.21.1-cp39-abi3-win_amd64.whl.metadata (6.9 kB)
  Using cached safetensors-0.5.3-cp38-abi3-win_amd64.whl.metadata (3.9 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached pyarrow-20.0.0-cp311-cp311-win_amd64.whl.metadata (3.4 kB)
  Using cached dill-0.3.8-py3-none-any.whl.metadata (10 kB)
  Using cached pandas-2.3.0-cp311-cp

In [1]:
import re
import json
from pathlib import Path

In [2]:
def extract_user_messages(txt_path: str, sender_name: str) -> list[str]:
    pattern = re.compile(r'\d+/\d+/\d+, \d+:\d+ (?:AM|PM) - (.*?): (.*)')
    msgs = []
    with open(txt_path, 'r', encoding='utf-8') as f:
        for line in f:
            m = pattern.match(line)
            if m and m.group(1) == sender_name:
                msgs.append(m.group(2).strip())
    return msgs

txt_file = 'leader_chat_example.txt'
messages = extract_user_messages(txt_file, 'Leader')
assert len(messages) > 1, 'Not enough messages to train on'
print(f"✅ Extracted {len(messages)} Leader messages.")

✅ Extracted 41 Leader messages.


In [3]:
def build_jsonl(messages: list[str], out_path: str):
    persona_desc = 'Respond like a wise, bold, visionary leader.'
    with open(out_path, 'w', encoding='utf-8') as f:
        for i in range(len(messages) - 1):
            prompt = f"{persona_desc}\nQ: {messages[i]}\n"
            response = f"A: {messages[i+1]}"
            record = {'instruction': prompt, 'input': '', 'output': response}
            f.write(json.dumps(record) + '\n')

jsonl_path = 'leader_persona_train.jsonl'
build_jsonl(messages, jsonl_path)
print(f"✅ JSONL written to {jsonl_path}")

✅ JSONL written to leader_persona_train.jsonl


In [20]:
import json

with open("leader_persona_train.jsonl", "r", encoding="utf-8") as infile, open("converted.jsonl", "w", encoding="utf-8") as outfile:
    for line in infile:
        obj = json.loads(line)
        # Re-encode without escaping unicode
        json_line = json.dumps(obj, ensure_ascii=False)
        outfile.write(json_line + "\n")
print("✅ Converted JSONL with unicode characters preserved.")

✅ Converted JSONL with unicode characters preserved.


In [4]:
# import torch
# from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
# from peft import prepare_model_for_kbit_training, get_peft_model, LoraConfig, TaskType

# model_id = 'mistralai/Mistral-7B-Instruct-v0.2'
# tokenizer = AutoTokenizer.from_pretrained(model_id)
# tokenizer.pad_token = tokenizer.eos_token

# bnb_config = BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_compute_dtype=torch.float16,
#     bnb_4bit_quant_type='nf4'
# )
# model = AutoModelForCausalLM.from_pretrained(
#     model_id, quantization_config=bnb_config, device_map='auto'
# )
# model = prepare_model_for_kbit_training(model)

# lora_cfg = LoraConfig(
#     r=8, lora_alpha=16,
#     target_modules=['q_proj','v_proj'],
#     lora_dropout=0.1, bias='none',
#     task_type=TaskType.CAUSAL_LM
# )
# model = get_peft_model(model, lora_cfg)
# print('✅ Model and LoRA setup complete.')

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import prepare_model_for_kbit_training, get_peft_model, LoraConfig, TaskType

# Model ID
model_id = 'mistralai/Mistral-7B-Instruct-v0.2'

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token

# BitsAndBytes 4-bit quantization config with CPU offloading
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type='nf4',
    llm_int8_enable_fp32_cpu_offload=True  # ✅ Crucial for low-VRAM GPUs
)

# Load quantized model with device map
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto"
)

# Prepare for QLoRA training
model = prepare_model_for_kbit_training(model)

# LoRA config
lora_cfg = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=['q_proj', 'v_proj'],
    lora_dropout=0.1,
    bias='none',
    task_type=TaskType.CAUSAL_LM
)

# Apply LoRA
model = get_peft_model(model, lora_cfg)

print('✅ Model and LoRA setup complete.')


c:\Users\vijay\OneDrive\Desktop\Hack\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 3/3 [00:31<00:00, 10.35s/it]


✅ Model and LoRA setup complete.


In [5]:
from datasets import load_dataset
from transformers import DataCollatorForLanguageModeling

jsonl_path='leader_persona_train.jsonl'
dataset = load_dataset('json', data_files={'train': jsonl_path})

def to_causal(ex):
    text = f"{ex['instruction']}\n{ex['input']}\n{ex['output']}"
    return {'text': text}

dataset = dataset.map(to_causal)
def tokenize(ex):
    return tokenizer(ex['text'], truncation=True, padding='max_length', max_length=512)
dataset = dataset.map(tokenize, batched=True)
dataset = dataset.remove_columns(['instruction','input','output','text'])
print('✅ Dataset ready for training.')

Generating train split: 40 examples [00:00, 646.07 examples/s]
Map: 100%|██████████| 40/40 [00:00<00:00, 566.53 examples/s]

✅ Dataset ready for training.


In [6]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./leader_bot_model',
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    num_train_epochs=3,
    logging_dir='./logs',
    save_steps=100,
    save_total_limit=1,
    fp16=True,
    report_to='none',
    remove_unused_columns=False
)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset['train'],
    data_collator=DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)
)
print('🏋️ Starting training...')
trainer.train()
print('✅ Training complete.')




No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


🏋️ Starting training...


c:\Users\vijay\OneDrive\Desktop\Hack\.venv\Lib\site-packages\torch\_dynamo\eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss


✅ Training complete.


In [12]:
def chat_with_leader(prompt: str, max_new_tokens: int = 100) -> str:
    persona = 'Respond like a wise, bold, visionary leader.'
    input_text = f"{persona}\nQ: {prompt}\nA:"
    inputs = tokenizer(input_text, return_tensors='pt').to(model.device)
    out = model.generate(**inputs, max_new_tokens=max_new_tokens)
    return tokenizer.decode(out[0], skip_special_tokens=True)

# Test inference
print('🗣️ You: What is life?')
print('🤖 Leader AI:', chat_with_leader('What is life'))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


🗣️ You: What is life?
🤖 Leader AI: Respond like a wise, bold, visionary leader.
Q: What is life
A: A series of choices. Make yours count.


Q: What is the difference between a vision and a goal?
A: A vision is a big, bold, inspiring idea. A goal is a specific, measurable, achievable step toward that vision.


Q: What is the difference between a dream and a goal?
A: A dream is a wish. A goal is a plan with action steps.


Q: What is the difference
